## SECCIÓN TABNET

#### PARTE 1: IMPORT DE LIBRERÍAS Y DATASET

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Cargamos los datos de entrenamiento y test desde archivos CSV. Como los datasets son muy grandes y tardan en cargarse, utilizamos una librería para paralelizar el código de carga.
# Librería utilizada: multiprocessing
from multiprocessing.pool import ThreadPool

# Función para cargar los archivos csv que tenemos como datasets.
def carregar(nom):
    return pd.read_csv(nom)

# Los csv que cargaremos son los siguientes:
csvs = ['fraudTrain.csv', 'fraudTest.csv']

# Creamos 2 grupos porque tenemoso dos archivos csv.
pool = ThreadPool(2)

# Cargamos los archivos en paralelo.
resultats = pool.map(carregar, csvs)

# Guardamos los resultados en sus respectivas variables de dataset de entrenamiento y test.
train_data = resultats[0]
test_data = resultats[1]

#### PARTE 2: DATA-PROCESSING

In [3]:
# En esta parte eliminamos la columna 'Unnamed: 0' que no aporta información relevante ya que es un índice
train_data = train_data.drop(columns='Unnamed: 0')
test_data = test_data.drop(columns='Unnamed: 0')

In [4]:
import time # Importamos la librería time para medir el tiempo de ejecución

# Convertimos la columna 'trans_date_trans_time' a formato timestamp para los dos conjuntos de datos y la columna 'dob' (date of birth) a formato timestamp para los dos conjuntos de datos
# En esta celda lo hacemos sin implementar paralelización para comparar el rendimiento con el mismo proceso pero con paralelización que hacemos en la siguiente celda
start_time_seq = time.time()

train_data['trans_date_trans_time'] = pd.to_datetime(train_data['trans_date_trans_time']).astype('int64') // 10**9 # Aquí convertimos la columna a timestamp en segundos mediante pandas, donde astype('int64') convierte la fecha a nanosegundos desde epoch y dividimos por 10**9 para obtener segundos
test_data['trans_date_trans_time'] = pd.to_datetime(test_data['trans_date_trans_time']).astype('int64') // 10**9 # Hacemos lo mismo para el conjunto de test

train_data['dob'] = pd.to_datetime(train_data['dob']).astype('int64') // 10**9 # Convertimos la columna 'dob' a timestamp en segundos
test_data['dob'] = pd.to_datetime(test_data['dob']).astype('int64') // 10**9 # Hacemos lo mismo para el conjunto de test

end_time_seq = time.time()
tiempo_seq = end_time_seq - start_time_seq
print(f"Tiempo Secuencial: {tiempo_seq:.4f} segundos")


Tiempo Secuencial: 1.0269 segundos


In [5]:
'''
# Hacemos lo mismo que en la celda anterior pero con paralelización para mejorar la velocidad
from multiprocessing import Pool, cpu_count

# Definimos una primera función que procese un chunk del DataFrame
def process_dates(df_chunk):
    if 'trans_date_trans_time' in df_chunk.columns:
        df_chunk['trans_date_trans_time'] = pd.to_datetime(df_chunk['trans_date_trans_time']).astype('int64') // 10**9 # Convertimos a timestamp en segundos de la fecha y hora de la transacción
    
    if 'dob' in df_chunk.columns:
        df_chunk['dob'] = pd.to_datetime(df_chunk['dob']).astype('int64') // 10**9 # Convertimos a timestamp en segundos de fecha de nacimiento
        
    return df_chunk

# Definimos una función para paralelizar el procesamiento del DataFrame
def parallelize_dataframe(df, func, n_cores=None):
    if n_cores is None: # Usamos todos los núcleos disponibles menos uno para no saturar el sistema
        n_cores = cpu_count() - 1 
    
    df_split = np.array_split(df, n_cores) # Dividimos el DataFrame en n_cores partes
    
    with Pool(n_cores) as pool: # Creamos un pool de procesos
        df = pd.concat(pool.map(func, df_split)) # Aplicamos la función a cada parte en paralelo y juntamos los resultados
        
    return df

start_time_par = time.time()

train_data = parallelize_dataframe(train_data, process_dates)
test_data = parallelize_dataframe(test_data, process_dates)

end_time_par = time.time()
tiempo_par = end_time_par - start_time_par
print(f"Tiempo Paralelo: {tiempo_par:.4f} segundos")
'''

'\n# Hacemos lo mismo que en la celda anterior pero con paralelización para mejorar la velocidad\nfrom multiprocessing import Pool, cpu_count\n\n# Definimos una primera función que procese un chunk del DataFrame\ndef process_dates(df_chunk):\n    if \'trans_date_trans_time\' in df_chunk.columns:\n        df_chunk[\'trans_date_trans_time\'] = pd.to_datetime(df_chunk[\'trans_date_trans_time\']).astype(\'int64\') // 10**9 # Convertimos a timestamp en segundos de la fecha y hora de la transacción\n\n    if \'dob\' in df_chunk.columns:\n        df_chunk[\'dob\'] = pd.to_datetime(df_chunk[\'dob\']).astype(\'int64\') // 10**9 # Convertimos a timestamp en segundos de fecha de nacimiento\n\n    return df_chunk\n\n# Definimos una función para paralelizar el procesamiento del DataFrame\ndef parallelize_dataframe(df, func, n_cores=None):\n    if n_cores is None: # Usamos todos los núcleos disponibles menos uno para no saturar el sistema\n        n_cores = cpu_count() - 1 \n\n    df_split = np.arra

In [6]:
'''
# Comprobamos la mejora de tiempo con paralelización
speedup = tiempo_seq / tiempo_par
print(f"El speedup es de: {speedup:.2f}x")
'''

'\n# Comprobamos la mejora de tiempo con paralelización\nspeedup = tiempo_seq / tiempo_par\nprint(f"El speedup es de: {speedup:.2f}x")\n'

#### PARTE 3: PREPARAMOS LOS DATOS Y INICIAMOS EL ENCODING

In [7]:
# Definimos una función que agrupa las tareas independientes para cada dataset.
def preprocesar_features(df):
    # En esta parte eliminamos las columnas que no aportan información relevante para este modelo
    df = df.drop(columns=['first', 'last', 'street', 'city', 'zip', 'job', 'merchant','merch_lat','merch_long','trans_num'])

    # Aquí aplicamos el one-hot encoding a las columnas categóricas, creando variables binarias para cada categoría, incrementando así la cantidad de columnas en ambos conjuntos de datos
    df = pd.get_dummies(df, columns=['gender', 'category', 'state'], drop_first=True)
    return df

# Creamos un grupo de 2 hilos para procesar las transformaciones de train y test simultáneamente.
pool_dummies = ThreadPool(2)
resultats_dummies = pool_dummies.map(preprocesar_features, [train_data, test_data])

# Recuperamos los datos procesados.
train_data, test_data = resultats_dummies[0], resultats_dummies[1]

# En esta parte alineamos las columnas del conjunto de test con las del conjunto de entrenamiento, por tal de tener el mismo númerod de columnas en ambos conjuntos de datos
# Pueden tener tamaños diferentes debido al one-hot encoding, ya que algunas categorías pueden estar presentes en el conjunto de training al ser más grande, pero no en el conjunto de test
train_cols = train_data.columns
test_data = test_data.reindex(columns=train_cols, fill_value=0)


#### PARTE 4: PROGRAMAMOS EL ALGORITMO DE TABNET (red neuronal + árbol de decisión)

In [8]:
# Importamos las librerías necesarias para el modelo TabNet
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split 

# Sepramos las características (X) de la variable objetivo (y) de los dos subgrupos
X_full = train_data.drop(columns='is_fraud').values
y_full = train_data['is_fraud'].values

X_test_final = test_data.drop(columns='is_fraud').values
y_test_final = test_data['is_fraud'].values

# Dividimos el set de datos para entrenamiento en training y validación, usando stratify para mantener la proporción de clases
X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.3, random_state=2003, stratify=y_full)

# Escalamos los datos usando RobustScaler
scaler = RobustScaler()
# Hacemos el fit y transformamos los datos de entrenamiento
X_train = scaler.fit_transform(X_train)
# Transformamos los datos de validación y test final usando el mismo scaler
X_val = scaler.transform(X_val)
X_test_final = scaler.transform(X_test_final)


# Creamos el modelo TabNet
clf = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,  # escogemos Adam como optimizador por su buen rendimiento
    optimizer_params=dict(lr=2e-2), # escogemos una tasa de aprendizaje inicial de 0.02
    scheduler_params={"step_size":10, "gamma":0.9}, # escogemos un scheduler que reduce la tasa de aprendizaje cada 10 epochs multiplicándola por 0.9
    scheduler_fn=torch.optim.lr_scheduler.StepLR, # escogemos StepLR como scheduler, lo cual sirve para reducir la tasa de aprendizaje durante el entrenamiento
    mask_type='entmax' # por ultimo, escogemos 'entmax' como tipo de máscara para el modelo TabNet, lo cual se utiliza para la atención en las características importantes
)

# Hacemos el fit del modelo 
clf.fit(
    X_train=X_train, y_train=y_train, # datos de entrenamiento
    eval_set=[(X_val, y_val)], # datos de validación para evaluar el modelo durante el entrenamiento
    eval_name=['val'], # nombre del conjunto de evaluación        
    eval_metric=['auc'], # como métrica de evaluación escogemos AUC (Area Under the Curve) porque es adecuada para problemas de clasificación binaria con clases desbalanceadas
    max_epochs=100, # número máximo de epochs para entrenar el modelo          
    patience=10, # si la métrica de evaluación no mejora durante 10 epochs, se detiene el entrenamiento temprano
    batch_size=1024, # tamaño del batch para el entrenamiento
    virtual_batch_size=128, # tamaño del batch virtual para la normalización por lotes
    num_workers=4, # número de trabajadores para cargar los datos y mejorar la paralelización
    drop_last=False, # si el último batch es más pequeño que el tamaño del batch, no se descarta
    weights=1 # no aplicamos pesos a las clases en este caso ya que TabNet maneja bien el desbalanceo
)

# Hacemos el predict y también obtenemos las probabilidades para el conjunto de validación
probs = clf.predict_proba(X_test_final)[:, 1]





c:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.18085 | val_auc: 0.96747 |  0:02:10s
epoch 1  | loss: 0.10812 | val_auc: 0.99137 |  0:04:26s
epoch 2  | loss: 0.0945  | val_auc: 0.99311 |  0:06:57s
epoch 3  | loss: 0.08679 | val_auc: 0.98711 |  0:09:09s
epoch 4  | loss: 0.08013 | val_auc: 0.99099 |  0:11:20s
epoch 5  | loss: 0.07303 | val_auc: 0.98948 |  0:13:29s
epoch 6  | loss: 0.0693  | val_auc: 0.98704 |  0:15:35s
epoch 7  | loss: 0.06591 | val_auc: 0.98608 |  0:17:41s
epoch 8  | loss: 0.06284 | val_auc: 0.98483 |  0:19:46s
epoch 9  | loss: 0.06103 | val_auc: 0.98302 |  0:21:50s
epoch 10 | loss: 0.05687 | val_auc: 0.97804 |  0:24:03s
epoch 11 | loss: 0.05593 | val_auc: 0.98592 |  0:26:14s
epoch 12 | loss: 0.0525  | val_auc: 0.97754 |  0:28:25s

Early stopping occurred at epoch 12 with best_epoch = 2 and best_val_auc = 0.99311


c:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


#### PARTE 5: EVALUACIÓN DE RESULTADOS

In [9]:
# En esta parte vamos a valorar los resultados obtenidos

# Ajuste del umbral para optimizar el recall utilizando las probabilidades obtenidas
threshhold = 0.3 # Ajustamos este threshhold según sea necesario, en este caso lo dejamos en 0.5 lo que significa que cualquier probabilidad por encima de 0.5 se clasifica como positiva (1).
# Si reducimos el umbral, aumentaríamos el recall pero podríamos disminuir la precisión, ya que más casos serían clasificados como positivos, es decir, más falsos positivos.
preds_recall_optimized = (probs >= threshhold).astype(int) # Convertimos las probabilidades a etiquetas binarias usando el umbral definido

# Calculamos la matriz de confusión para obtener TP, FP, TN, FN
cm = confusion_matrix(y_test_final, preds_recall_optimized)
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
TP = cm[1, 1]
fraudes_totales = FN + TP

# Evaluamos el modelo con el umbral optimizado y hacemos un print para ver los resultados
print(f"Classification Report (Threshold > {threshhold}):")
print(classification_report(y_test_final, preds_recall_optimized)) # Reporte de clasificación que incluye precisión, recall y F1-score

print(f"Verdaderos positivos (TP, Fraudes detectados): {TP}")
print(f"Falsos positivos (FP, Transacciones legítimas detectadas como fraude): {FP}")
print(f"Falsos Negativos (FN, Fraudes no detectados): {FN}")
print(f"Casos de fraudes totales: {fraudes_totales}")
print(f"Recall: {TP / fraudes_totales:.4f}")
print(f"Ratio de falsos positivos (FPR): {FP / (FP + TN):.4f}")


Classification Report (Threshold > 0.3):
              precision    recall  f1-score   support

           0       1.00      0.95      0.97    553574
           1       0.07      0.95      0.13      2145

    accuracy                           0.95    555719
   macro avg       0.53      0.95      0.55    555719
weighted avg       1.00      0.95      0.97    555719

Verdaderos positivos (TP, Fraudes detectados): 2031
Falsos positivos (FP, Transacciones legítimas detectadas como fraude): 27129
Falsos Negativos (FN, Fraudes no detectados): 114
Casos de fraudes totales: 2145
Recall: 0.9469
Ratio de falsos positivos (FPR): 0.0490


In [10]:
import joblib

# 1. Guardar el model TabNet
# Això crearà un fitxer 'tabnet_model.zip'
clf.save_model('tabnet_model')

# 2. Guardar el Scaler (RobustScaler)
# És vital per escalar les dades noves igual que les d'entrenament
joblib.dump(scaler, 'scaler.pkl')

# 3. Guardar les COLUMNES d'entrenament (CRÍTIC)
# Necessitem saber quines columnes exactes van entrar al model després del get_dummies
# per assegurar que l'app web tingui la mateixa estructura.
model_columns = list(train_data.drop(columns='is_fraud').columns)
joblib.dump(model_columns, 'model_columns.pkl')

print("Arxius generats correctament: tabnet_model.zip, scaler.pkl, model_columns.pkl")

Successfully saved model at tabnet_model.zip
Arxius generats correctament: tabnet_model.zip, scaler.pkl, model_columns.pkl
